In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import os
import math
import time

# Description

## 1. Statistics
- Number of vehicles
- Tightness of time window 

In [2]:
Homberger = glob.glob("instances/Homberger/*.TXT")
Solomon = glob.glob("instances/Solomon/*.txt")

In [3]:
file_list = Homberger + Solomon

In [4]:
#file = "instances/Homberger/C1_2_1.TXT"
file = "instances/toy.txt"

In [5]:
def stats_instance(file):
    content = []
    f = open(file,"r")
    for x in f:
        content.append(x)
    instance_name = content[0] #name of the instance
    V, Q = content[4].split() #V is the number of vehicles, Q is the vehicle capacity.
    V = int(V); Q = int(Q)
    header = ['CUST NO.', 'XCOORD.', 'YCOORD.', 'DEMAND', 'READY TIME', 'DUE DATE', 'SERVICE TIME']
    df = pd.DataFrame([x.split() for x in content[9:]], columns=header)
    col_list = ['XCOORD.', 'YCOORD.', 'DEMAND', 'READY TIME', 'DUE DATE', 'SERVICE TIME']
    df[col_list] = df[col_list].astype('int')
    return V, Q, df

In [6]:
V, Q, df = stats_instance(file)

In [7]:
df

,CUST NO.,XCOORD.,YCOORD.,DEMAND,READY TIME,DUE DATE,SERVICE TIME
0,0,50,50,0,0,200,0
1,1,35,65,10,45,50,10
2,2,30,55,30,50,80,10
3,3,40,45,10,0,20,10
4,4,55,70,20,0,100,10
5,5,75,50,10,50,70,10
6,6,54,35,20,17,20,10


In [8]:
df['CUST NO.'] = range(1, len(df)+1)

In [9]:
def make_path(directory_name): # make path if not exist
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
make_path('data/map'); make_path('data/requests'); make_path('data/vehicles')

In [10]:
# extract nodes
df[['CUST NO.', 'XCOORD.', 'YCOORD.']].to_csv('data/map/nodes.csv', header=False, index=False) 
# TODO: we encode cust no+1, it needs decode later

In [11]:
# extract edges
nodes_df = df[['CUST NO.', 'XCOORD.', 'YCOORD.']].set_index('CUST NO.')

In [12]:
def Euclidean(n1, n2): # euclidean distances truncated to one decimal place 
    x1,y1 = n1; x2,y2 = n2
    return round(math.sqrt((x1 - x2)**2+ (y1 - y2)**2),1)

In [13]:
edges_df = []
node_list = sorted(df['CUST NO.'].to_list())
times_df = pd.DataFrame(index=node_list, columns = node_list)
for i in node_list:
    for j in node_list:
        dist = Euclidean(nodes_df.loc[i],nodes_df.loc[j])
        times_df[i][j] = dist
        if i != j:
            edges_df.append([i, j, dist])
pd.DataFrame(edges_df).to_csv('data/map/edges.csv', header=False, index=False)
times_df.to_csv('data/map/times.csv', header=False, index=False)

In [14]:
# vehicle should start at the depot
depot_lat, depot_lon = df[['XCOORD.', 'YCOORD.']].iloc[0]


In [15]:
vehicle_df = []
for id in range(V):
    # columns are [id, start node, lat, lon, start time, capacity]
    vehicle_df.append([id, 1, depot_lat, depot_lon, time.strftime('%H:%M:%S', time.gmtime(0)), Q])
pd.DataFrame(vehicle_df).to_csv('data/vehicles/vehicles.csv', header=False, index=False)

In [31]:
request_column = ['pickup_id', 'pickup_x', 'pickup_y', 'dropoff_id', 'dropoff_x', 'dropoff_y', 'request_time', 
                  'waiting_time', 'demand']

In [28]:
df

,CUST NO.,XCOORD.,YCOORD.,DEMAND,READY TIME,DUE DATE,SERVICE TIME
0,1,50,50,0,0,200,0
1,2,35,65,10,45,50,10
2,3,30,55,30,50,80,10
3,4,40,45,10,0,20,10
4,5,55,70,20,0,100,10
5,6,75,50,10,50,70,10
6,7,54,35,20,17,20,10


In [32]:
# columns are [request_id, pickup_id, pickup_x, pickup_y, 
# dropoff_id, dropoff_x, dropoff_y, request_time]
request_df = []
for i in range(1, len(df)):
    request_df.append([df['CUST NO.'].iloc[i], df['XCOORD.'].iloc[i], df['YCOORD.'].iloc[i], 
    1, df['XCOORD.'].iloc[0], df['YCOORD.'].iloc[0], time.strftime('%H:%M:%S', time.gmtime(df['READY TIME'].iloc[i])),
    df['DUE DATE'].iloc[i]-df['READY TIME'].iloc[i], df['DEMAND'].iloc[i]])

In [33]:
pd.DataFrame(request_df, columns = request_column).to_csv('data/requests/requests.csv', header=False)

## 2.  Translate CVRPTW instances to simulator inputs. 
- Set vehicle limit to NUMBER
- Set carsize to CAPACITY
- Set pickup time to READY TIME and pickup location to the customer location (XCOORD., YCOORD.)
- Set dropoff location to the depot
- Set requests **specific waiting time** to (DUE DATE- READY TIME)
- Set maximum detour time to be large enough value (DUE DATE of the depot)
- Set dwell time (alighting) to SERVICE TIME (it is constant in an instance)
- **Need to select interval**
- set DUE DATE of the depot to FINAL_TIME 

## 3. Lists of modifying simulator
### 3.1. Set requests specific waiting time
### 3.2. Set requests specific demands quantity $q_i$   
- Generate $q_i$ separate requests (if the demand of each customer is allowed to be served by multiple vehicles).  
- **Update the number of passengers in the vehicle by increasing $q_i$** (if not allowed).    